In [76]:
import cv2
import numpy as np
from math import acos, pi, sqrt, cos, e

In [147]:
#Mean, Gaussian, Median Filter 모두 같은 마스크 크기 적용
#사각형이라고 가정, 보통은 정사각형이라 3*3, 5*5, 7*7을 기준으로 생각할 것
maskheight = int(input("Height of square : "))
maskwidth = int(input("Width of square : "))
A = float(input("A (the number that multiply Original image) : "))

Height of square : 3
Width of square : 3
A (the number that multiply Original image) : 1


In [148]:
# B,G,R
def compute_Hue(B, G, R):
    # prevent divide by 0, adding 0.00000001
    angle = 0.5*((R - G) + (R - B)) / sqrt((R-G)*(R-G) + (R-B)*(G-B) + 0.00000001)
    if B<= G:
        return acos(angle)
    elif B> G:
        return (2*pi- acos(angle))
    #return acos(angle) if B <= G else (2*pi - acos(angle))

In [149]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 포함
def mask_highpass_1(src, height, width, maskheight, maskwidth):
    
    I = np.zeros((height, width))
    S = np.zeros((height, width))
    H = np.zeros((height, width))


    #B, G, R = cv2.split(src)
    B = src[:,:,0]
    G = src[:,:,1]
    R = src[:,:,2]


    for i in range(height):
        for j in range(width):
            B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
            I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
            # prevent divide by 0, adding 0.00000001
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
            H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)
    
    
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 다이아몬드 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight // 2 + A
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
                
            elif (j <= temp and j >= maskwidth-temp-1) or (j >= temp and j <= maskwidth-temp-1) :
                mask[i][j] = -1
                
            else:
                mask[i][j] = 0
            
        # m이 maskheight 중간 지점을 넘어가기 이전에는 masking 되는 부분을 2개씩 늘어나게 하기
        if i < (maskheight+1)//2 -1 :                 
            temp += 1
                
            
        # m이 maskheight 중간 지점을 넘어가면 masking 되는 부분을 2개씩 줄어들게 하기
        else :    # m > (maskwidth+1)//2
            temp -= 1
        
    print(mask)
    
    
    #meanfilter 함수를 불러왔을 때 자동으로 hsi도 선언
    hsi2B = np.zeros((height, width))
    hsi2G = np.zeros((height, width))
    hsi2R = np.zeros((height, width))
        
    for i in range(height):
        for j in range(width):
            
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                    hsi2B[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2R[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2G[i][j] = ((I[i][j]*3. - (hsi2R[i][j]+hsi2B[i][j])))
            
                if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                    H[i][j] = H[i][j]-2*pi/3
                    hsi2R[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2G[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2B[i][j] = ((I[i][j]*3 - (hsi2R[i][j]+hsi2G[i][j])))
        
                if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                    H[i][j] = H[i][j] - 4*pi/3
                    hsi2G[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2B[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2R[i][j] = ((I[i][j]*3 - (hsi2G[i][j]+hsi2B[i][j])))
                
                continue
            
            temp_I = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp_I = (I[i+m][j+n] * mask[m][n])
                        
                    if H[i+m][j+n] >= 0 and H[i+m][j+n] < 2*pi/3: #0도~120도
                        hsi2B[i][j] += (1-S[i+m][j+n])*temp_I
                        hsi2R[i][j] += (1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I
                        hsi2G[i][j] += (temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I)))
            
                    if H[i+m][j+n] >= 2*pi/3 and H[i+m][j+n] < 4*pi/3:
                        H[i+m][j+n] = H[i+m][j+n]-2*pi/3
                        hsi2R[i][j] += (1-S[i+m][j+n])*temp_I
                        hsi2G[i][j] += (1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I
                        hsi2B[i][j] += (temp_I*3 - (((1-S[i+m][j+n])*temp_I)+((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)))
        
                    if H[i+m][j+n] >= 4*pi/3 and H[i+m][j+n] <= 2*pi:
                        H[i+m][j+n] = H[i+m][j+n] - 4*pi/3
                        hsi2G[i][j] += (1-S[i+m][j+n])*temp_I
                        hsi2B[i][j] += (1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I
                        hsi2R[i][j] += (temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I)))
            
    plt = cv2.merge((hsi2B, hsi2G, hsi2R))
    
    return plt

In [150]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 포함
def mask_highpass_2(src, height, width, maskheight, maskwidth):
    
    I = np.zeros((height, width))
    S = np.zeros((height, width))
    H = np.zeros((height, width))


    #B, G, R = cv2.split(src)
    B = src[:,:,0]
    G = src[:,:,1]
    R = src[:,:,2]


    for i in range(height):
        for j in range(width):
            B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
            I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
            # prevent divide by 0, adding 0.00000001
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
            H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)
    
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 사각형 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight-1 + A
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
            else:
                mask[i][j] = -1
            
    print(mask)
    
    
    #meanfilter 함수를 불러왔을 때 자동으로 hsi도 선언
    hsi2B = np.zeros((height, width))
    hsi2G = np.zeros((height, width))
    hsi2R = np.zeros((height, width))
        
    for i in range(height):
        for j in range(width):
            
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                    hsi2B[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2R[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2G[i][j] = ((I[i][j]*3. - (hsi2R[i][j]+hsi2B[i][j])))
            
                if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                    H[i][j] = H[i][j]-2*pi/3
                    hsi2R[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2G[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2B[i][j] = ((I[i][j]*3 - (hsi2R[i][j]+hsi2G[i][j])))
        
                if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                    H[i][j] = H[i][j] - 4*pi/3
                    hsi2G[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2B[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2R[i][j] = ((I[i][j]*3 - (hsi2G[i][j]+hsi2B[i][j])))
                
                continue
            
            temp_I = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp_I = (I[i+m][j+n] * mask[m][n])
                        
                    if H[i+m][j+n] >= 0 and H[i+m][j+n] < 2*pi/3: #0도~120도
                        hsi2B[i][j] += (1-S[i+m][j+n])*temp_I
                        hsi2R[i][j] += (1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I
                        hsi2G[i][j] += (temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I)))
            
                    if H[i+m][j+n] >= 2*pi/3 and H[i+m][j+n] < 4*pi/3:
                        H[i+m][j+n] = H[i+m][j+n]-2*pi/3
                        hsi2R[i][j] += (1-S[i+m][j+n])*temp_I
                        hsi2G[i][j] += (1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I
                        hsi2B[i][j] += (temp_I*3 - (((1-S[i+m][j+n])*temp_I)+((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)))
        
                    if H[i+m][j+n] >= 4*pi/3 and H[i+m][j+n] <= 2*pi:
                        H[i+m][j+n] = H[i+m][j+n] - 4*pi/3
                        hsi2G[i][j] += (1-S[i+m][j+n])*temp_I
                        hsi2B[i][j] += (1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I
                        hsi2R[i][j] += (temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I)))
            
    plt = cv2.merge((hsi2B, hsi2G, hsi2R))
    
    return plt

In [177]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 포함
#mask가 mean filter일 때
def mask_lowpass_1(src, height, width, maskheight, maskwidth):
    
    I = np.zeros((height, width))
    S = np.zeros((height, width))
    H = np.zeros((height, width))


    #B, G, R = cv2.split(src)
    B = src[:,:,0]
    G = src[:,:,1]
    R = src[:,:,2]


    for i in range(height):
        for j in range(width):
            B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
            I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
            # prevent divide by 0, adding 0.00000001
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
            H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)
    
    
    
    #meanfilter는 mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.float32)
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    pixel_num = maskheight*maskwidth
    
    #mean filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
    for i in range(maskheight):
        for j in range(maskwidth):
            mask[i][j] = -1/pixel_num 
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] += A
        
    print(mask)
    
    
    #meanfilter 함수를 불러왔을 때 자동으로 hsi도 선언
    hsi2B = np.zeros((height, width))
    hsi2G = np.zeros((height, width))
    hsi2R = np.zeros((height, width))
        
    for i in range(height):
        for j in range(width):
            
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                    hsi2B[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2R[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2G[i][j] = ((I[i][j]*3. - (hsi2R[i][j]+hsi2B[i][j])))
            
                if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                    H[i][j] = H[i][j]-2*pi/3
                    hsi2R[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2G[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2B[i][j] = ((I[i][j]*3 - (hsi2R[i][j]+hsi2G[i][j])))
        
                if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                    H[i][j] = H[i][j] - 4*pi/3
                    hsi2G[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2B[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2R[i][j] = ((I[i][j]*3 - (hsi2G[i][j]+hsi2B[i][j])))
                
                continue
            
            temp_I = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp_I = (1/pixel_num*I[i+m][j+n] * mask[m][n])
                        
                    if H[i+m][j+n] >= 0 and H[i+m][j+n] < 2*pi/3: #0도~120도
                        hsi2B[i][j] += (1-S[i+m][j+n])*temp_I
                        hsi2R[i][j] += (1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I
                        hsi2G[i][j] += (temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I)))
            
                    if H[i+m][j+n] >= 2*pi/3 and H[i+m][j+n] < 4*pi/3:
                        H[i+m][j+n] = H[i+m][j+n]-2*pi/3
                        hsi2R[i][j] += (1-S[i+m][j+n])*temp_I
                        hsi2G[i][j] += (1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I
                        hsi2B[i][j] += (temp_I*3 - (((1-S[i+m][j+n])*temp_I)+((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)))
        
                    if H[i+m][j+n] >= 4*pi/3 and H[i+m][j+n] <= 2*pi:
                        H[i+m][j+n] = H[i+m][j+n] - 4*pi/3
                        hsi2G[i][j] += (1-S[i+m][j+n])*temp_I
                        hsi2B[i][j] += (1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I
                        hsi2R[i][j] += (temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I)))
            
    plt = cv2.merge((hsi2B, hsi2G, hsi2R))
    
    return plt

In [178]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 포함
def mask_lowpass_2(src, height, width, maskheight, maskwidth):
    
    I = np.zeros((height, width))
    S = np.zeros((height, width))
    H = np.zeros((height, width))


    #B, G, R = cv2.split(src)
    B = src[:,:,0]
    G = src[:,:,1]
    R = src[:,:,2]


    for i in range(height):
        for j in range(width):
            B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
            I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
            # prevent divide by 0, adding 0.00000001
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
            H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)
    
    
    #gausianfilter는 mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.float32)
    
    #결과값
    plt = np.zeros((height, width,3), dtype=np.uint8)

    
    #sigma 입력받기. sigma가 커질 수록 분포가 넓어짐
    sigma = int(input("가우시안 필터의 시그마 값을 정수로 입력해주세요 : "))

    g_sum = 0
    #gaussian filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
    for i in range(maskheight):
        for j in range(maskwidth):
            mask[i][j] = -(1/(2*pi*sigma*sigma)) * e ** (-((i**2) + (j**2))/(sigma*sigma))
            g_sum += mask[i][j]
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] += A
    
            
    print(mask)
    
    
    #meanfilter 함수를 불러왔을 때 자동으로 hsi도 선언
    hsi2B = np.zeros((height, width))
    hsi2G = np.zeros((height, width))
    hsi2R = np.zeros((height, width))
        
    for i in range(height):
        for j in range(width):
            
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                    hsi2B[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2R[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2G[i][j] = ((I[i][j]*3. - (hsi2R[i][j]+hsi2B[i][j])))
            
                if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                    H[i][j] = H[i][j]-2*pi/3
                    hsi2R[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2G[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2B[i][j] = ((I[i][j]*3 - (hsi2R[i][j]+hsi2G[i][j])))
        
                if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                    H[i][j] = H[i][j] - 4*pi/3
                    hsi2G[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2B[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2R[i][j] = ((I[i][j]*3 - (hsi2G[i][j]+hsi2B[i][j])))
                
                continue
            
            temp_I = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp_I = (1/g_sum * I[i+m][j+n] * mask[m][n])
                        
                    if H[i+m][j+n] >= 0 and H[i+m][j+n] < 2*pi/3: #0도~120도
                        hsi2B[i][j] += (1-S[i+m][j+n])*temp_I
                        hsi2R[i][j] += (1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I
                        hsi2G[i][j] += (temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I)))
            
                    if H[i+m][j+n] >= 2*pi/3 and H[i+m][j+n] < 4*pi/3:
                        H[i+m][j+n] = H[i+m][j+n]-2*pi/3
                        hsi2R[i][j] += (1-S[i+m][j+n])*temp_I
                        hsi2G[i][j] += (1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I
                        hsi2B[i][j] += (temp_I*3 - (((1-S[i+m][j+n])*temp_I)+((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)))
        
                    if H[i+m][j+n] >= 4*pi/3 and H[i+m][j+n] <= 2*pi:
                        H[i+m][j+n] = H[i+m][j+n] - 4*pi/3
                        hsi2G[i][j] += (1-S[i+m][j+n])*temp_I
                        hsi2B[i][j] += (1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I
                        hsi2R[i][j] += (temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I)))
            
    plt = cv2.merge((hsi2B, hsi2G, hsi2R))
    
    return plt

In [179]:
src = cv2.imread('./cat.jpg', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
        
highpass1 = np.zeros((height, width,3), dtype=np.uint8)
highpass2 = np.zeros((height, width,3), dtype=np.uint8)
lowpass1 = np.zeros((height, width,3), dtype=np.uint8)
lowpass2 = np.zeros((height, width,3), dtype=np.uint8)

highpass1 = mask_highpass_1(src, height, width, maskheight, maskwidth)
highpass2 = mask_highpass_2(src, height, width, maskheight, maskwidth)
lowpass1 = mask_lowpass_1(src, height, width, maskheight, maskwidth)
lowpass1 = mask_lowpass_2(src, height, width, maskheight, maskwidth)
cv2.imshow("Using High Pass Filter 1", highpass1)
cv2.imshow("Using High Pass Filter 2", highpass2)
cv2.imshow("Using Low Pass Filter 1", lowpass1)
cv2.imshow("Using Low Pass Filter 2", lowpass2)
#cv2.imshow("Gaussian Filter", gaussian)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[ 0. -1.  0.]
 [-1.  5. -1.]
 [ 0. -1.  0.]]
[[-1. -1. -1.]
 [-1.  9. -1.]
 [-1. -1. -1.]]
[[-0.11111111 -0.11111111 -0.11111111]
 [-0.11111111  0.8888889  -0.11111111]
 [-0.11111111 -0.11111111 -0.11111111]]
가우시안 필터의 시그마 값을 정수로 입력해주세요 : 3
[[-0.01768388 -0.01582423 -0.01133856]
 [-0.01582423  0.98583984 -0.01014619]
 [-0.01133856 -0.01014619 -0.00727006]]


In [ ]:
------------------------------------------------

In [26]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 미포함
def mask_highpass_1(src, height, width, maskheight, maskwidth):
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 다이아몬드 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight // 2
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
                
            elif (j <= temp and j >= maskwidth-temp-1) or (j >= temp and j <= maskwidth-temp-1) :
                mask[i][j] = -1
                
            else:
                mask[i][j] = 0
            
        # m이 maskheight 중간 지점을 넘어가기 이전에는 masking 되는 부분을 2개씩 늘어나게 하기
        if i < (maskheight+1)//2 -1 :                 
            temp += 1
                
            
        # m이 maskheight 중간 지점을 넘어가면 masking 되는 부분을 2개씩 줄어들게 하기
        else :    # m > (maskwidth+1)//2
            temp -= 1
      
    print(mask)
    
    R = np.array(src[:,:,0])
    G = np.array(src[:,:,1])
    B = np.array(src[:,:,2])
    
    R = R*0.299
    G = G*0.587
    B = B*0.114
    
    avg = R+G+B
    gray = np.zeros((height, width, 3), dtype=np.uint8)
    for i in range(3):
        gray[:,:,i] = avg
    
    for i in range(height):
        for j in range(width):
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                plt[i][j] = gray[i][j]
                continue
            
            temp = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp +=  (A*gray[i+m][j+n] + mask[m][n])/pixel_sum
            
            
            plt[i][j] =  temp
            
    plt += src
        
    return plt


In [20]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 미포함
def mask_highpass_1(src, height, width, maskheight, maskwidth):
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 다이아몬드 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight // 2
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
                
            elif (j <= temp and j >= maskwidth-temp-1) or (j >= temp and j <= maskwidth-temp-1) :
                mask[i][j] = -1
                
            else:
                mask[i][j] = 0
            
        # m이 maskheight 중간 지점을 넘어가기 이전에는 masking 되는 부분을 2개씩 늘어나게 하기
        if i < (maskheight+1)//2 -1 :                 
            temp += 1
                
            
        # m이 maskheight 중간 지점을 넘어가면 masking 되는 부분을 2개씩 줄어들게 하기
        else :    # m > (maskwidth+1)//2
            temp -= 1
      
    print(mask)
    
    R = np.array(src[:,:,0])
    G = np.array(src[:,:,1])
    B = np.array(src[:,:,2])
    
    R = R*0.299
    G = G*0.587
    B = B*0.114
    
    avg = R+G+B
    gray = np.zeros((height, width, 3), dtype=np.uint8)
    for i in range(3):
        gray[:,:,i] = avg
    
    for i in range(height):
        for j in range(width):
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                plt[i][j] = gray[i][j]
                continue
            
            temp = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp +=  (gray[i+m][j+n] * mask[m][n])/pixel_sum
            
            
            plt[i][j] =  temp
            
    #plt += src
        
    return plt


In [23]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 미포함
def mask_highpass_1(src, height, width, maskheight, maskwidth):
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 다이아몬드 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight // 2
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
                
            elif (j <= temp and j >= maskwidth-temp-1) or (j >= temp and j <= maskwidth-temp-1) :
                mask[i][j] = -1
                
            else:
                mask[i][j] = 0
            
        # m이 maskheight 중간 지점을 넘어가기 이전에는 masking 되는 부분을 2개씩 늘어나게 하기
        if i < (maskheight+1)//2 -1 :                 
            temp += 1
                
            
        # m이 maskheight 중간 지점을 넘어가면 masking 되는 부분을 2개씩 줄어들게 하기
        else :    # m > (maskwidth+1)//2
            temp -= 1
      
    print(mask)
    
    for i in range(height):
        for j in range(width):
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                plt[i][j] = src[i][j]
                continue
            
            temp = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp +=  ((A+1)*src[i+m][j+n] + mask[m][n])/pixel_sum
            plt[i][j] =  temp
         
    #plt += src
    
    return plt


In [61]:
def mask_highpass_1(src, height, width, maskheight, maskwidth):
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 다이아몬드 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight // 2
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
                
            elif (j <= temp and j >= maskwidth-temp-1) or (j >= temp and j <= maskwidth-temp-1) :
                mask[i][j] = -1
                
            else:
                mask[i][j] = 0
            
        # m이 maskheight 중간 지점을 넘어가기 이전에는 masking 되는 부분을 2개씩 늘어나게 하기
        if i < (maskheight+1)//2 -1 :                 
            temp += 1
                
            
        # m이 maskheight 중간 지점을 넘어가면 masking 되는 부분을 2개씩 줄어들게 하기
        else :    # m > (maskwidth+1)//2
            temp -= 1
      
    plt = cv2.filter2D(src, -1, mask)
    plt += src
    
    return plt

In [27]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 미포함
def mask_highpass_2(src, height, width, maskheight, maskwidth):
    
     # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 사각형 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight-1
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
            else:
                mask[i][j] = -1
            
    print(mask)
    
    R = np.array(src[:,:,0])
    G = np.array(src[:,:,1])
    B = np.array(src[:,:,2])
    
    R = R*0.299
    G = G*0.587
    B = B*0.114
    
    avg = R+G+B
    gray = np.zeros((height, width, 3), dtype=np.uint8)
    for i in range(3):
        gray[:,:,i] = avg
    
    for i in range(height):
        for j in range(width):
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                plt[i][j] = gray[i][j]
                continue
            
            temp = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp +=  (A*gray[i-m][j-n] + mask[m][n])/pixel_sum
            
            
            plt[i][j] =  temp
            
    plt += src
        
    return plt


In [21]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 미포함
def mask_highpass_2(src, height, width, maskheight, maskwidth):
    
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 사각형 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight-1
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
            else:
                mask[i][j] = -1
            
    print(mask)
    
    R = np.array(src[:,:,0])
    G = np.array(src[:,:,1])
    B = np.array(src[:,:,2])
    
    R = R*0.299
    G = G*0.587
    B = B*0.114
    
    avg = R+G+B
    gray = np.zeros((height, width, 3), dtype=np.uint8)
    for i in range(3):
        gray[:,:,i] = avg
    
    for i in range(height):
        for j in range(width):
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                plt[i][j] = gray[i][j]
                continue
            
            temp = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp +=  (gray[i+m][j+n] * mask[m][n])/pixel_sum
            
            
            plt[i][j] =  temp
            
    #plt += src
        
    return plt


In [24]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 미포함
def mask_highpass_2(src, height, width, maskheight, maskwidth):
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 사각형 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight-1
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
            else:
                mask[i][j] = -1
            
    print(mask)
    
    for i in range(height):
        for j in range(width):
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                plt[i][j] = src[i][j]
                continue
            
            temp = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp +=  ((A+1)*src[i+m][j+n] + mask[m][n])/pixel_sum
            plt[i][j] =  temp
    
    #plt += src
    
    return plt


In [62]:
def mask_highpass_2(src, height, width, maskheight, maskwidth):
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 사각형 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight-1
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
            else:
                mask[i][j] = -1
            
    print(mask)
      
    plt = cv2.filter2D(src, -1, mask)
    plt += src
    
    return plt

In [63]:
src = cv2.imread('./Fig0327(a)(tungsten_original).jpg', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
        
highpass1 = np.zeros((height, width,3), dtype=np.uint8)
highpass2 = np.zeros((height, width,3), dtype=np.uint8)
lowpass = np.zeros((height, width,3), dtype=np.uint8)

highpass1 = mask_highpass_1(src, height, width, maskheight, maskwidth)
highpass2 = mask_highpass_2(src, height, width, maskheight, maskwidth)
#lowpass = gaussianfilter(src, height, width, maskheight, maskwidth)
cv2.imshow("Using High Pass Filter 1", highpass1)
cv2.imshow("Using High Pass Filter 2", highpass2)
#cv2.imshow("Gaussian Filter", gaussian)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[-1. -1. -1.]
 [-1.  8. -1.]
 [-1. -1. -1.]]


In [937]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 미포함
def mask_highpass_1(src, height, width, maskheight, maskwidth):
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 다이아몬드 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight // 2 + A
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
                
            elif (j <= temp and j >= maskwidth-temp-1) or (j >= temp and j <= maskwidth-temp-1) :
                mask[i][j] = -1
                
            else:
                mask[i][j] = 0
            
        # m이 maskheight 중간 지점을 넘어가기 이전에는 masking 되는 부분을 2개씩 늘어나게 하기
        if i < (maskheight+1)//2 -1 :                 
            temp += 1
                
            
        # m이 maskheight 중간 지점을 넘어가면 masking 되는 부분을 2개씩 줄어들게 하기
        else :    # m > (maskwidth+1)//2
            temp -= 1
      
    print(mask)
    
    for i in range(height):
        for j in range(width):
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                plt[i][j] = src[i][j]
                continue
            
            temp = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp +=  mask[m][n] * (src[i+m][j+n] + 1)
            plt[i][j] =  temp
            
    return plt


In [950]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 미포함
def mask_highpass_2(src, height, width, maskheight, maskwidth):
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 다이아몬드 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight // 2 + A
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
                
            else:
                mask[i][j] = -1
            
    print(mask)
    
    for i in range(height):
        for j in range(width):
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                plt[i][j] = src[i][j]
                continue
            
            temp = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp +=  mask[m][n] * src[i+m][j+n]
            plt[i][j] =  temp
            
    return plt


In [928]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 포함
def mask_highpass_1(src, height, width, maskheight, maskwidth):
    
    I = np.zeros((height, width))
    S = np.zeros((height, width))
    H = np.zeros((height, width))


    #B, G, R = cv2.split(src)
    B = src[:,:,0]
    G = src[:,:,1]
    R = src[:,:,2]


    for i in range(height):
        for j in range(width):
            B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
            I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
            # prevent divide by 0, adding 0.00000001
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
            H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)
    
    
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 다이아몬드 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight // 2 + A
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
                
            elif (j <= temp and j >= maskwidth-temp-1) or (j >= temp and j <= maskwidth-temp-1) :
                mask[i][j] = -1
                
            else:
                mask[i][j] = 0
            
        # m이 maskheight 중간 지점을 넘어가기 이전에는 masking 되는 부분을 2개씩 늘어나게 하기
        if i < (maskheight+1)//2 -1 :                 
            temp += 1
                
            
        # m이 maskheight 중간 지점을 넘어가면 masking 되는 부분을 2개씩 줄어들게 하기
        else :    # m > (maskwidth+1)//2
            temp -= 1
        
    print(mask)
    
    
    #meanfilter 함수를 불러왔을 때 자동으로 hsi도 선언
    hsi2B_mean = np.zeros((height, width))
    hsi2G_mean = np.zeros((height, width))
    hsi2R_mean = np.zeros((height, width))
        
    for i in range(height):
        for j in range(width):
            
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                    hsi2B_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2R_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2G_mean[i][j] = ((I[i][j]*3. - (hsi2R_mean[i][j]+hsi2B_mean[i][j])))
            
                if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                    H[i][j] = H[i][j]-2*pi/3
                    hsi2R_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2G_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2B_mean[i][j] = ((I[i][j]*3 - (hsi2R_mean[i][j]+hsi2G_mean[i][j])))
                    H[i][j] = H[i][j]+2*pi/3
        
                if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                    H[i][j] = H[i][j] - 4*pi/3
                    hsi2G_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2B_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2R_mean[i][j] = ((I[i][j]*3 - (hsi2G_mean[i][j]+hsi2B_mean[i][j])))
                    H[i][j] = H[i][j] + 4*pi/3
                
                continue
            
            temp_I, temp_B, temp_G, temp_R = 0, 0, 0, 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp_I =  ((A+1)*src[i+m][j+n] + mask[m][n])/pixel_sum
            
                    if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                        temp_B += ((1-S[i+m][j+n])*temp_I)
                        temp_R += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        temp_G += ((temp_I*3. - (temp_R+temp_B)))
            
                    if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                        H[i][j] = H[i][j]-2*pi/3
                        temp_R += ((1-S[i+m][j+n])*temp_I)
                        temp_G += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        temp_B += ((temp_I*3 - (temp_R+temp_G)))
                        H[i][j] = H[i][j]+2*pi/3
        
                    if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                        H[i][j] = H[i][j] - 4*pi/3
                        temp_G += ((1-S[i+m][j+n])*temp_I)
                        temp_B += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        temp_R += ((temp_I*3 - (temp_G+temp_B)))
                        H[i][j] = H[i][j] + 4*pi/3
            
            hsi2B_mean[i][j], hsi2G_mean[i][j], hsi2R_mean[i][j] = temp_B, temp_G, temp_R
            
    plt = cv2.merge((hsi2B_mean, hsi2G_mean, hsi2R_mean))
    
    return plt

In [495]:
# histogram - low
def histogram_low(value):
    height, width = value.shape[0], value.shape[1]
    value_low = 255
    value_high = 0
    for i in range(height):
        for j in range(width):
            if value_low >= value[i][j]:
                value_low = value[i][j]
            if value_high <= value[i][j]:
                value_high = value[i][j]
    
    for i in range(height):
        for j in range(width):
            if value[i][j] <= value_low:
                value[i][j] = 0
            elif value[i][j] > value_low and value[i][j] < value_high:
                value[i][j] = 255*(value[i][j] - value_low)/(value_high - value_low)
            else: #value[i][j] >= value_high
                value[i][j] = 255
                
    return value

In [500]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 포함
def mask_highpass_1(src, height, width, maskheight, maskwidth):
    
    I = np.zeros((height, width))
    S = np.zeros((height, width))
    H = np.zeros((height, width))


    #B, G, R = cv2.split(src)
    B = src[:,:,0]
    G = src[:,:,1]
    R = src[:,:,2]


    for i in range(height):
        for j in range(width):
            B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
            I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
            # prevent divide by 0, adding 0.00000001
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
            H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)
    
    
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    gray = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 다이아몬드 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight // 2
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
                
            elif (j <= temp and j >= maskwidth-temp-1) or (j >= temp and j <= maskwidth-temp-1) :
                mask[i][j] = -1
                
            else:
                mask[i][j] = 0
            
        # m이 maskheight 중간 지점을 넘어가기 이전에는 masking 되는 부분을 2개씩 늘어나게 하기
        if i < (maskheight+1)//2 -1 :                 
            temp += 1
                
            
        # m이 maskheight 중간 지점을 넘어가면 masking 되는 부분을 2개씩 줄어들게 하기
        else :    # m > (maskwidth+1)//2
            temp -= 1
        
    print(mask)

    
    #meanfilter 함수를 불러왔을 때 자동으로 hsi도 선언
    hsi2B_mean = np.zeros((height, width))
    hsi2G_mean = np.zeros((height, width))
    hsi2R_mean = np.zeros((height, width))
        
    for i in range(height):
        for j in range(width):
            
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                    hsi2B_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2R_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2G_mean[i][j] = ((I[i][j]*3. - (hsi2R_mean[i][j]+hsi2B_mean[i][j])))
            
                if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                    H[i][j] = H[i][j]-2*pi/3
                    hsi2R_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2G_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2B_mean[i][j] = ((I[i][j]*3 - (hsi2R_mean[i][j]+hsi2G_mean[i][j])))
        
                if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                    H[i][j] = H[i][j] - 4*pi/3
                    hsi2G_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2B_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2R_mean[i][j] = ((I[i][j]*3 - (hsi2G_mean[i][j]+hsi2B_mean[i][j])))
                
                continue
            
            temp_I, temp_B, temp_G, temp_R = 0, 0, 0, 0
            m, n = (maskheight-1)//2, (maskwidth-1)//2
            temp_I = ((A+1)*I[i+m][j+n] + mask[m][n])/pixel_sum
            
            if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                temp_B += ((1-S[i+m][j+n])*temp_I)
                temp_R += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                temp_G += ((temp_I*3. - (temp_R+temp_B)))
            
            if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                H[i][j] = H[i][j]-2*pi/3
                temp_R += ((1-S[i+m][j+n])*temp_I)
                temp_G += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                temp_B += ((temp_I*3 - (temp_R+temp_G)))
        
            if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                H[i][j] = H[i][j] - 4*pi/3
                temp_G += ((1-S[i+m][j+n])*temp_I)
                temp_B += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                temp_R += ((temp_I*3 - (temp_G+temp_B)))
            
            #hsi2B_mean[i][j], hsi2G_mean[i][j], hsi2R_mean[i][j] = temp_B, temp_G, temp_R
            
            hsi2B_mean[i][j], hsi2G_mean[i][j], hsi2R_mean[i][j] = temp_B, temp_G, temp_R
            
    avg = (hsi2B_mean * 0.114 + hsi2G_mean * 0.587 + hsi2R_mean * 0.299)
    
    gray = cv2.merge((avg, avg, avg))
    #plt = cv2.merge((hsi2B_mean, hsi2G_mean, hsi2R_mean))
    
    print(gray)
    return gray

In [ ]:
#결론!

In [46]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 포함
def mask_highpass_1(src, height, width, maskheight, maskwidth):
    
    I = np.zeros((height, width))
    S = np.zeros((height, width))
    H = np.zeros((height, width))


    #B, G, R = cv2.split(src)
    B = src[:,:,0]
    G = src[:,:,1]
    R = src[:,:,2]


    for i in range(height):
        for j in range(width):
            B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
            I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
            # prevent divide by 0, adding 0.00000001
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
            H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)
    
    
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 다이아몬드 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight // 2 + A
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
                
            elif (j <= temp and j >= maskwidth-temp-1) or (j >= temp and j <= maskwidth-temp-1) :
                mask[i][j] = -1
                
            else:
                mask[i][j] = 0
            
        # m이 maskheight 중간 지점을 넘어가기 이전에는 masking 되는 부분을 2개씩 늘어나게 하기
        if i < (maskheight+1)//2 -1 :                 
            temp += 1
                
            
        # m이 maskheight 중간 지점을 넘어가면 masking 되는 부분을 2개씩 줄어들게 하기
        else :    # m > (maskwidth+1)//2
            temp -= 1
        
    print(mask)
    
    
    #meanfilter 함수를 불러왔을 때 자동으로 hsi도 선언
    hsi2B_mean = np.zeros((height, width))
    hsi2G_mean = np.zeros((height, width))
    hsi2R_mean = np.zeros((height, width))
        
    for i in range(height):
        for j in range(width):
            
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                    hsi2B_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2R_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2G_mean[i][j] = ((I[i][j]*3. - (hsi2R_mean[i][j]+hsi2B_mean[i][j])))
            
                if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                    H[i][j] = H[i][j]-2*pi/3
                    hsi2R_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2G_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2B_mean[i][j] = ((I[i][j]*3 - (hsi2R_mean[i][j]+hsi2G_mean[i][j])))
                    H[i][j] = H[i][j]+2*pi/3
        
                if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                    H[i][j] = H[i][j] - 4*pi/3
                    hsi2G_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2B_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2R_mean[i][j] = ((I[i][j]*3 - (hsi2G_mean[i][j]+hsi2B_mean[i][j])))
                    H[i][j] = H[i][j] + 4*pi/3
                
                continue
            
            temp_I = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp_I = (I[i+m][j+n] * mask[m][n])
                        
                    if H[i+m][j+n] >= 0 and H[i+m][j+n] < 2*pi/3: #0도~120도
                        hsi2B_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2R_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2G_mean[i][j] += ((temp_I*3. - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I))))
            
                    if H[i+m][j+n] >= 2*pi/3 and H[i+m][j+n] < 4*pi/3:
                        H[i+m][j+n] = H[i+m][j+n]-2*pi/3
                        hsi2R_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2G_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2B_mean[i][j] += ((temp_I*3 - (((1-S[i+m][j+n])*temp_I)+((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I))))
                        H[i+m][j+n] = H[i+m][j+n]+2*pi/3
        
                    if H[i+m][j+n] >= 4*pi/3 and H[i+m][j+n] <= 2*pi:
                        H[i+m][j+n] = H[i+m][j+n] - 4*pi/3
                        hsi2G_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2B_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2R_mean[i][j] += ((temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I))))
                        H[i+m][j+n] = H[i+m][j+n] + 4*pi/3
            
    plt = cv2.merge((hsi2B_mean, hsi2G_mean, hsi2R_mean))
    
    return plt

In [47]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 포함
def mask_highpass_2(src, height, width, maskheight, maskwidth):
    
    I = np.zeros((height, width))
    S = np.zeros((height, width))
    H = np.zeros((height, width))


    #B, G, R = cv2.split(src)
    B = src[:,:,0]
    G = src[:,:,1]
    R = src[:,:,2]


    for i in range(height):
        for j in range(width):
            B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
            I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
            # prevent divide by 0, adding 0.00000001
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
            H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)
    
    
    # mask라 사이즈 지정 잘 해주기
    mask = np.zeros((maskheight, maskwidth), dtype=np.uint8)/255
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    
    # 사각형 형태의 mask
    temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
    center = maskwidth * maskheight-1 + A
    pixel_sum = maskwidth*maskheight
    for i in range(maskheight):
        for j in range(maskwidth):
            if (i == (maskheight+1)//2-1 and j == (maskwidth+1)//2-1):
                mask[i][j] = center
            else:
                mask[i][j] = -1
            
    print(mask)
    
    
    #meanfilter 함수를 불러왔을 때 자동으로 hsi도 선언
    hsi2B_mean = np.zeros((height, width))
    hsi2G_mean = np.zeros((height, width))
    hsi2R_mean = np.zeros((height, width))
        
    for i in range(height):
        for j in range(width):
            
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                    hsi2B_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2R_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2G_mean[i][j] = ((I[i][j]*3. - (hsi2R_mean[i][j]+hsi2B_mean[i][j])))
            
                if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                    H[i][j] = H[i][j]-2*pi/3
                    hsi2R_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2G_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2B_mean[i][j] = ((I[i][j]*3 - (hsi2R_mean[i][j]+hsi2G_mean[i][j])))
                    H[i][j] = H[i][j]+2*pi/3
        
                if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                    H[i][j] = H[i][j] - 4*pi/3
                    hsi2G_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2B_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2R_mean[i][j] = ((I[i][j]*3 - (hsi2G_mean[i][j]+hsi2B_mean[i][j])))
                    H[i][j] = H[i][j] + 4*pi/3
                
                continue
            
            temp_I = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp_I = (I[i+m][j+n] * mask[m][n])
                        
                    if H[i+m][j+n] >= 0 and H[i+m][j+n] < 2*pi/3: #0도~120도
                        hsi2B_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2R_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2G_mean[i][j] += ((temp_I*3. - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I))))
            
                    if H[i+m][j+n] >= 2*pi/3 and H[i+m][j+n] < 4*pi/3:
                        H[i+m][j+n] = H[i+m][j+n]-2*pi/3
                        hsi2R_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2G_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2B_mean[i][j] += ((temp_I*3 - (((1-S[i+m][j+n])*temp_I)+((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I))))
                        H[i+m][j+n] = H[i+m][j+n]+2*pi/3
        
                    if H[i+m][j+n] >= 4*pi/3 and H[i+m][j+n] <= 2*pi:
                        H[i+m][j+n] = H[i+m][j+n] - 4*pi/3
                        hsi2G_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2B_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2R_mean[i][j] += ((temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I))))
                        H[i+m][j+n] = H[i+m][j+n] + 4*pi/3
            
    plt = cv2.merge((hsi2B_mean, hsi2G_mean, hsi2R_mean))
    
    return plt